In [1]:
import os
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
from torchsummary import summary
import torch.utils.data as utils
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
from torch.utils.data import Dataset
from tqdm import tqdm
import h5py

In [2]:
Comp = 'FabioPC'
if Comp == 'FabioPC':
    df = pd.read_csv('/media/fabio/Disco locale/Fabio/Programmazione/Python/Poliambulanza/Alzheimer/Dati/ADNI/ADNI_TOT_1_10_2020.csv')
else:
    df = pd.read_csv('D:/ADNI/Dati/ADNI1_T1_2_11_2020.csv')

In [3]:
df.head()

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,1270076,002_S_5178,SMC,M,75,103,MRI,HighResHippocampus,Original,11/25/2019,DCM,NaN
1,1270075,002_S_5178,SMC,M,75,103,MRI,Sagittal 3D FLAIR,Original,11/25/2019,DCM,NaN
2,1270074,002_S_5178,SMC,M,75,103,MRI,Field Mapping,Original,11/25/2019,DCM,NaN
3,1270072,002_S_5178,SMC,M,75,103,MRI,Axial 3TE T2 STAR,Original,11/25/2019,DCM,NaN
4,1270071,002_S_5178,SMC,M,75,103,MRI,Axial 3TE T2 STAR,Original,11/25/2019,DCM,NaN


In [4]:
interpolator = sitk.sitkLinear
default_value = 0
print('*Translation Transform*')
transform = sitk.TranslationTransform(3)

*Translation Transform*


In [5]:
if Comp == 'FabioPC':
    IMG_Folder = '/media/fabio/Disco locale/Fabio/Programmazione/Python/Poliambulanza/Alzheimer/Dati/ADNI/ADNI_Prova/'
else:
    IMG_Folder = 'D:/ADNI/Dati/ADNI_T1/ADNI1_T1/ADNI_Registrate/'

In [6]:
dati_img = os.listdir(IMG_Folder)
cwd = os.getcwd()

In [7]:
cwd

'/media/fabio/Disco locale/Fabio/Programmazione/Python/Poliambulanza/Alzheimer/ADNI/LeNet5_CNN'

In [8]:
dati_img[1:]

['ADNI_002_S_0295_MR_MPR-R__GradWarp_Br_20070319114720743_S13407_I45114 (10).nii',
 'ADNI_002_S_0295_MR_MPR-R__GradWarp_Br_20070319114720743_S13407_I45114 (11).nii',
 'ADNI_002_S_0295_MR_MPR-R__GradWarp_Br_20070319114720743_S13407_I45114 (12).nii',
 'ADNI_002_S_0295_MR_MPR-R__GradWarp_Br_20070319114720743_S13407_I45114 (13).nii',
 'ADNI_002_S_0295_MR_MPR-R__GradWarp_Br_20070319114720743_S13407_I45114 (14).nii',
 'ADNI_002_S_0295_MR_MPR-R__GradWarp_Br_20070319114720743_S13407_I45114 (15).nii',
 'ADNI_002_S_0295_MR_MPR-R__GradWarp_Br_20070319114720743_S13407_I45114 (2).nii',
 'ADNI_002_S_0295_MR_MPR-R__GradWarp_Br_20070319114720743_S13407_I45114 (3).nii',
 'ADNI_002_S_0295_MR_MPR-R__GradWarp_Br_20070319114720743_S13407_I45114 (4).nii',
 'ADNI_002_S_0295_MR_MPR-R__GradWarp_Br_20070319114720743_S13407_I45114 (5).nii',
 'ADNI_002_S_0295_MR_MPR-R__GradWarp_Br_20070319114720743_S13407_I45114 (6).nii',
 'ADNI_002_S_0295_MR_MPR-R__GradWarp_Br_20070319114720743_S13407_I45114 (7).nii',
 'ADNI_002

In [9]:
if Comp == 'FabioPC':
    dati_img[0].split('_')[1:4]
else:
    dati_img[0].split('_')[2:5]

In [10]:
if Comp == 'FabioPC':
    ID = '_'.join(dati_img[0].split('_')[1:4])
else:
    ID = '_'.join(dati_img[0].split('_')[2:5])

In [11]:
ID

'002_S_0295'

In [12]:
df.loc[df['Subject'] == ID]['Modality'].values[0]

'MRI'

In [13]:
reference_image = sitk.ReadImage(IMG_Folder + dati_img[1])

In [14]:
img_tot = []
y = []
try:
    os.makedirs('ADNI1_PROC')
except:
    print("Already Created")
for data_img in tqdm(dati_img[1:]):
    if 'nii' in str(data_img):
        print(str(data_img))
        sitk_t1 = sitk.ReadImage(IMG_Folder + data_img)
        #print(sitk_t1.GetDirection())
        #t2 = sitk.Resample(sitk_t1, reference_image, transform,interpolator, default_value)
        t1 = sitk.GetArrayFromImage(sitk_t1)
        img_tot.append(t1)
        print()
        if Comp == 'FabioPC':
            Subject = df.loc[df['Subject'] == '_'.join(data_img.split('_')[1:4])]['Group'].values[0]
        else:
            Subject = df.loc[df['Subject'] == '_'.join(data_img.split('_')[2:5])]['Group'].values[0]
        print(Subject)
        y.append(Subject)
        #sitk.WriteImage(t2,'ADNI1_PROC/'+ str(data_img))

        print(t1.shape)

  0%|          | 0/77 [00:00<?, ?it/s]

Already Created
ADNI_002_S_0295_MR_MPR-R__GradWarp_Br_20070319114720743_S13407_I45114 (10).nii

CN
(166, 256, 256)
ADNI_002_S_0295_MR_MPR-R__GradWarp_Br_20070319114720743_S13407_I45114 (11).nii


  3%|▎         | 2/77 [00:00<00:19,  3.88it/s]


CN
(166, 256, 256)
ADNI_002_S_0295_MR_MPR-R__GradWarp_Br_20070319114720743_S13407_I45114 (12).nii


  4%|▍         | 3/77 [00:00<00:22,  3.22it/s]

...


CN
(166, 256, 256)
ADNI_002_S_0413_MR_MPR__GradWarp__B1_Correction_Br_20070319115531324_S13893_I45118 (8).nii


 99%|█████████▊| 76/77 [00:46<00:00,  1.05it/s]


CN
(166, 256, 256)
ADNI_002_S_0413_MR_MPR__GradWarp__B1_Correction_Br_20070319115531324_S13893_I45118 (9).nii


100%|██████████| 77/77 [00:47<00:00,  1.00it/s]


CN
(166, 256, 256)


In [36]:
num_chunks = 77
N = len(img_tot)
chunks = np.linspace(0, N, num_chunks)

In [37]:
chunks = [int(chunk) for chunk in chunks]

In [53]:
try:
    if Comp == 'FabioPC':
        os.makedirs('/media/fabio/Disco locale/Fabio/Programmazione/Python/Poliambulanza/Alzheimer/Dati/ADNI/ADNI_Prova/H5Corrette')
    else:
        os.makedirs('D:/ADNI/Dati/ADNI_T1/ADNI1_T1/ADNI_Registrate/H5Corrette')
except:
    print('Già creata')

Già creata


In [90]:
if Comp == 'FabioPC':
    hf = h5py.File('/media/fabio/Disco locale/Fabio/Programmazione/Python/Poliambulanza/Alzheimer/Dati/ADNI/ADNI_Prova/H5Corrette/data_img.h5', 'w')
else:
    hf = h5py.File('D:/ADNI/Dati/ADNI_T1/ADNI1_T1/ADNI_Registrate/H5Corrette/data_img.h5', 'w')
img_data = hf.create_dataset('data', (1, 166, 256, 256), maxshape=(None, None, None, None))
label_data = hf.create_dataset('label', (1,1), maxshape=(None, None))
for i in range(len(chunks) -1):
    grp = hf.create_group("group_{}".format(i))
    
    print(i)
    sub_img_tot = img_tot[chunks[i]:chunks[i+1]]
    label = y[chunks[i]:chunks[i+1]]
    
    label = label_encoder.fit_transform(label)
    enc = OneHotEncoder()

    # 2. FIT
    enc.fit(label.reshape(-1, 1))

    # 3. Transform
    label = enc.transform(label.reshape(-1, 1)).toarray()
    print(label)
    sub_img_array = np.asarray(sub_img_tot)
    print(sub_img_array.shape)
    img_data.resize((i+1, 166, 256, 256))
    label_data.resize((i+1,1))
    img_data[i,:,:,:] = sub_img_array
    label_data[i] = label
hf.close()

0
[[1.]]
(1, 166, 256, 256)


/home/fabio/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


1
[[1.]]
(1, 166, 256, 256)
2
[[1.]]
(1, 166, 256, 256)


/home/fabio/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/fabio/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can speci

3
[[1.]]
(1, 166, 256, 256)
4
[[1.]]
(1, 166, 256, 256)


...

34
[[1.]]
(1, 166, 256, 256)
35
[[1.]]
(1, 166, 256, 256)


/home/fabio/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/fabio/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can speci

36
[[1.]]
(1, 256, 256, 166)


/home/fabio/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


TypeError: Can't broadcast (1, 256, 256, 166) -> (166, 256, 256)

In [91]:
hf.close()

In [ ]:
#date = np.asarray(img_tot)

In [ ]:

y = label_encoder.fit_transform(y)
enc = OneHotEncoder()

# 2. FIT
enc.fit(y.reshape(-1, 1))

# 3. Transform
y = onehotlabels = enc.transform(y.reshape(-1, 1)).toarray()



In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(date, y, test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
plt.imshow(X_train[0][:, 100, :], cmap='gray')

In [ ]:
class CustomDataset(Dataset):
    """Arguments:
        img (tensor)
        label (tensor)
    """

    def __init__(self, img, label):
    
        self.img = img
        self.label = label
        
    def __len__(self):
        return len(self.label)

    def __getitem__(self, index):
        img = self.img[index]
        
        label = self.label[index]
        return img, label


In [ ]:
t2 = torch.from_numpy(Y_train)
t1 = torch.from_numpy(X_train)
dataset = CustomDataset(t1, t2)
dataloader = utils.DataLoader(dataset, batch_size=1, num_workers=8, shuffle=True)

In [46]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv3d(1, 10, kernel_size=(5, 5, 5))
        self.conv2 = nn.Conv3d(10, 20, kernel_size=(5, 5, 5))
        self.conv2_drop = nn.Dropout3d()
        self.fc1 = nn.Linear(38 * 61 * 61 * 20, 50)
        self.fc2 = nn.Linear(50,3)
        
    def forward(self, x):
        x = F.relu(F.max_pool3d(self.conv1(x), (2, 2, 2)))
        x = F.relu(F.max_pool3d(self.conv2_drop(self.conv2(x)), (2, 2, 2)))
        x = x.view(-1, 38 * 61 * 61 * 20)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
model = Net().to(device)
summary(model, (1, 52, 92, 80))

RuntimeError: CUDA out of memory. Tried to allocate 540.00 MiB (GPU 0; 1.95 GiB total capacity; 421.48 MiB already allocated; 330.88 MiB free; 256.52 MiB cached)

In [42]:
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=1e-5, weight_decay=1e-5)

In [43]:
#x = torch.randn(2, 1, 192, 192, 160, device='cuda:0')

In [44]:
from torch.autograd import Variable

In [45]:
x = Variable(torch.randn(10, 1, 52, 92, 80)).to(torch.device("cuda" if torch.cuda.is_available() else "cpu")) # (N,C,D,H,W)
# print(x)
y = model(x)
print(y.shape)

torch.Size([10, 3])


In [ ]:
label = torch.randn(10,3)

In [ ]:
for i in range(0,10):
    loss_list, batch_list = [], []

    #for num, (img, label) in enumerate(dataloader):
        #inputs = img.unsqueeze(0)
    print()
    output = model(x.float().to(torch.device("cuda" if torch.cuda.is_available() else "cpu")))
    loss = criterion(output, label.float().to(torch.device("cuda" if torch.cuda.is_available() else "cpu")))
    optimizer.zero_grad()
    loss_list.append(loss.detach().cpu().item())
    batch_list.append(i+1)
    loss.backward()
    optimizer.step()
    print('Train - Epoch %d, Batch: %d, Loss: %f' % (i, num, loss.detach().cpu().item()))

In [ ]:
''' SCRIPT DA USARE PER 3D SLICER 

##################################################
##################################################
##################################################

>>> scene = slicer.mrmlScene
>>> slicer.util.selectModule('BRAINSFit')
>>> brainsFit = slicer.modules.brainsfit

>>> [ success,movingVolumeNode ] = slicer.util.loadVolume('/media/fabio/Disco locale/Scaricati/ADNI1_ex/ADNI/002_S_0559/MPR-R__GradWarp__B1_Correction__N3/2006-05-23_15_09_38.0/S14876/ADNI_002_S_0559_MR_MPR-R__GradWarp__B1_Correction__N3_Br_20070216235035331_S14876_I40678.nii',returnNode=True)
>>> movingVolumeNode
(MRMLCorePython.vtkMRMLScalarVolumeNode)0x7f5738020d50
>>> success
True

>>> [ success, fixedVolumeNode ] = slicer.util.loadVolume('/media/fabio/Disco locale/Scaricati/ADNI1_ex/ADNI/011_S_0008/MPR-R__GradWarp__B1_Correction__N3/2005-09-13_13_33_53.0/S9195/ADNI_011_S_0008_MR_MPR-R__GradWarp__B1_Correction__N3_Br_20061208113724677_S9195_I32254.nii', returnNode = True)
>>> success
True
>>> parameterRigid = {}
>>> linearTransform = slicer.vtkMRMLLinearTransformNode()
>>> parameterRigid["linearTransform"] = linearTransform.GetID()
>>> parameterRigid["useRigid"] = True
>>> slicer.cli.run(brainsFit, None, parameterRigid)
parameter  linearTransform  has unsupported type  NoneType
(MRMLCLIPython.vtkMRMLCommandLineModuleNode)0x7f5738020e20
>>> parameterRigid["fixedVolume"] = fixedVolumeNode
>>> parameterRigid["movingVolume"] = movingVolumeNode
>>> slicer.cli.run(brainsFit, None, parameterRigid)
parameter  linearTransform  has unsupported type  NoneType
(MRMLCLIPython.vtkMRMLCommandLineModuleNode)0x7f5738030328
>>> slicer.mrmlScene.AddNode( linearTransform )
(MRMLCorePython.vtkMRMLLinearTransformNode)0x7f5738020e88
>>> slicer.cli.run(brainsFit, None, parameterRigid)
parameter  linearTransform  has unsupported type  NoneType
(MRMLCLIPython.vtkMRMLCommandLineModuleNode)0x7f5738020db8
>>> parameterRigid["linearTransform"] = linearTransform.GetID()
>>> slicer.cli.run(brainsFit, None, parameterRigid)
(MRMLCLIPython.vtkMRMLCommandLineModuleNode)0x7f5738030328
>>> a = slicer.cli.run(brainsFit, None, parameterRigid)
>>> a
(MRMLCLIPython.vtkMRMLCommandLineModuleNode)0x7f5738020e20
>>> a.GetStatusString()
'Completed'
>>> linearTransform = slicer.vtkMRMLLinearTransformNode()
>>> a = slicer.cli.run(brainsFit, None, parameterRigid)
>>> a.GetStatusString()
'Completed'
>>> parameterRigid["useRigid"] = True
>>> parameterRigid["samplingPercentage"] = 0.002
>>> a = slicer.cli.run(brainsFit, None, parameterRigid)
>>> parameterRigid["samplingPercentage"] = 0.02
>>> a = slicer.cli.run(brainsFit, None, parameterRigid)
>>> a.GetStatus()
2

>>> a = slicer.cli.run(brainsFit, None, parameterRigid)
>>> a.GetStatus()
2
>>> a.GetStatus()
32
>>> parameterRigid["samplingPercentage"] = 0.002
>>> a = slicer.cli.run(brainsFit, None, parameterRigid)
>>> parameterRigid["samplingPercentage"] = 0.02
>>> a = slicer.cli.run(brainsFit, None, parameterRigid)
>>> parameterRigid["samplingPercentage"] = 0.002
>>> 
>>> parameterRigid["initializeTransformMode"] = "useCenterOfHead"
>>> a = slicer.cli.run(brainsFit, None, parameterRigid)
>>> a.GetStatus()
96
>>> a.GetStatus()
96
>>> a.GetStatus()
96
>>> a.GetStatus()
96
>>> a.GetStatusString()
'Completed with errors'
>>> a.GetErrorText()
"General Registration (BRAINS) standard error:\n\nPARSE ERROR: Argument: (--initializeTransformMode)\n             Value 'useCenterOfHead' does not meet constraint: Off|useMomentsAlign|useCenterOfHeadAlign|useGeometryAlign|useCenterOfROIAlign\n\nBrief USAGE: \n\nFor complete USAGE and HELP type: \n   /media/fabio/Disco locale/Scaricati/Slicer-4.10.2-linux-amd64/bin/../lib/Slicer-4.10/cli-modules/BRAINSFit --help\n\n"
>>> parameterRigid["initializeTransformMode"] = "useCenterOfHeadAlign"
>>> a = slicer.cli.run(brainsFit, None, parameterRigid)
>>> a.GetStatusString()
'Running'
>>> a.GetStatus()
2
>>> a.GetStatusString()
'Running'
>>> a.GetStatus()
2
>>> a.GetStatus()
2
>>> a.GetStatusString()
'Running'
>>> a.GetStatusString()
'Running'
>>> a.GetErrorText()
''
>>> a.GetStatusString()
'Completed'
>>> a.GetStatusString()
'Completed'
>>> 
>>> 
>>> parameterRigid["samplingPercentage"] = 0.02
>>> a = slicer.cli.run(brainsFit, None, parameterRigid)
>>> a.GetStatusString()
'Running'
>>> a.GetStatusString()
'Running'
>>> a.GetStatusString()
'Completed'
>>> 
'''